## Naive RAG
### organized_data klasöründen en basit haliyle cevap veriyor.

In [ ]:
# Gerekli kütüphanelerin eklenmesi
import os
import getpass
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# .env dosyasını yükleyerek API anahtarlarını getir
load_dotenv() 

# API Anahtarları
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
#os.environ['LANGCHAIN_TRACING_V2'] = 'true'
#os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

# Model Tanımlaması
model = ChatOpenAI(model="gpt-4")
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

#### INDEXING ####

# Ana veri dizini: Tüm kategoriler altındaki dosyaları yükleme
base_directory = "rag_data/website/organized_data"

# DirectoryLoader ile belirtilen dizindeki tüm alt klasörlerden txt dosyalarını yükle
loader = DirectoryLoader(base_directory, glob="**/*.txt", loader_cls=TextLoader)

# Belgeleri yükle
docs = loader.load()

# Embedding işlemi: Text Splitter gerekmediği için belgeleri direkt olarak embed ediyoruz
vectorstore = Chroma.from_documents(documents=docs, 
                                    embedding=embedding)

# Retriever tanımlaması
retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt: Hub'dan prompt çekiliyor
custom_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(
        """
        You are a Telekom-Hilfe assistant for question-answering tasks, providing answers to Telekom customers or potential customers.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise.
        Question: {question}
        Context: {context}
        Answer:
        """
    )
])

# Belgeleri formatlama fonksiyonu
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain tanımlaması
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)

# Soru: Zinciri çalıştırarak soruyu soruyoruz
result = rag_chain.invoke("Magenta tv hakkinda bilgi verir misin?")
print(result)